<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-1%20%EA%B2%B0%EC%A0%95%20%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Testing with real world data

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
import datetime
import os, re

In [24]:
from xgboost import XGBClassifier
import xgboost
from lightgbm import LGBMClassifier
import lightgbm
import joblib

In [3]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, SGDRegressor
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
import pickle

# write list, dictionary to pickle
def save_to_pickle(path, filename):
    open_file = open(path, "wb")
    pickle.dump(filename, open_file)
    open_file.close()

# read list, dictionary from pickle
def load_from_pickle(path):
    open_file = open(path, "rb")
    loaded_file = pickle.load(open_file)
    open_file.close()
    return loaded_file

In [5]:
import csv

# write list, dictionary to csv
# path = './xxx/', my_dict = filename

def save_dict_to_csv(path, my_dict):
    df = pd.DataFrame.from_dict(my_dict, orient='index') 
    df.to_csv (path, index=False, header=True)  
    
def save_list_to_csv(path, my_list):
    df = pd.DataFrame(my_list, columns=['columns'])
    df.to_csv (path, index=False, header=True) 

In [6]:
def to_df(date, com_name, precision, y_predict, weight):
    dict_temp = {}
    dict_temp['date'] = date
    dict_temp[f'{com_name}_precision'] = f'{precision:.2f}'
    dict_temp[f'{com_name}_predict'] = f'{y_predict[0]}'
    dict_temp[f'{com_name}_yes'] = f'{weight[0,0]:.2f}'
    dict_temp[f'{com_name}_no'] = f'{weight[0,1]:.2f}'
    df_t = pd.DataFrame.from_dict(dict_temp, orient='index').T
    df_t.set_index('date', inplace=True)
    return df_t

In [30]:
code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy'], 
        '000660' : ['SK하이닉스', 'skhinix'], '207940' : ['삼성바이오로직스', 'ssbio'],
        '006400' : ['삼성SDI', 'sdi'], '051910' : ['LG화학', 'lgchemical'],
        '005935' : ['삼성전자우', 'secpre'], '005380' : ['현대차', 'hyunmotor'],
        '035420' : ['NAVER', 'naver'], '000270' : ['기아','kia'],
        '035720' : ['카카오', 'kakao'], '005490' : ['POSCO홀딩스', 'poscoholding'],
        '105560' : ['KB금융', 'kbbank'], '028260' : ['삼성물산', 'sscnt'],
        '068270' : ['셀트리온', 'celltrion'], '012330' : ['현대모비스', 'mobis'],
        '055550' : ['신한지주', 'shgroup'], '066570' : ['LG전자', 'lgelec'],
        '003670' : ['포스코케미칼', 'poscochemical'], '096770' : ['SK이노베이션', 'skinnovation'],
        '033780' : ['KT&G', 'ktng'], '030200' : ['KT', 'kt']}

code_good = {'005930' : ['삼성전자', 'sec'], '035420' : ['NAVER', 'naver'],
             '035720' : ['카카오', 'kakao'], '012330' : ['현대모비스', 'mobis'],
             '051910' : ['LG화학', 'lgchemical'], '005935' : ['삼성전자우', 'secpre'],
             '373220' : ['LG에너지솔루션', 'lgenergy'],
#              '005380' : ['현대차', 'hyunmotor'] # to bad code.
            }

code_good = {'005930' : ['삼성전자', 'sec'], 
            }

code_mid = {'105560' : ['KB금융', 'kbbank'],
            '003670' : ['포스코케미칼', 'poscochemical'], '006400' : ['삼성SDI', 'sdi'],
            }

code_bad = { '000270' : ['기아','kia'], '030200' : ['KT', 'kt'],
             '033780' : ['KT&G', 'ktng'], '066570' : ['LG전자', 'lgelec'], 
             '005490' : ['POSCO홀딩스', 'poscoholding'], '055550' : ['신한지주', 'shgroup'],
             '000660' : ['SK하이닉스', 'skhinix'], '096770' : ['SK이노베이션', 'skinnovation'],
             '207940' : ['삼성바이오로직스', 'ssbio'], '028260' : ['삼성물산', 'sscnt'],
             '068270' : ['셀트리온', 'celltrion'],
           }

In [33]:
directory_for_predict = '../data/data_for_ml/predict/'

if not os.path.exists(directory_for_predict+ 'prediction/prediction_list.pkl'):
    os.makedirs(directory_for_predict+'prediction')
    prediction_list=pd.DataFrame()
    fname_p = 'prediction_list.pkl'
    path_p = directory_for_predict+'prediction/' + fname_p
    prediction_list.to_pickle(path_p)

In [40]:
prediction_list = pd.read_pickle(directory_for_predict+ 'prediction/prediction_list.pkl')

In [41]:
prediction_list.head()

,sec_precision,sec_predict,sec_yes,sec_no,naver_precision,naver_predict,naver_yes,naver_no,kakao_precision,kakao_predict,...,lgchemical_yes,lgchemical_no,secpre_precision,secpre_predict,secpre_yes,secpre_no,lgenergy_precision,lgenergy_predict,lgenergy_yes,lgenergy_no
date,,,,,,,,,,,,,,,,,,,,,
2023-04-03,0.93,0,0.52,0.48,1.00,1,0.48,0.52,1.00,0,...,0.50,0.50,1.00,0,0.62,0.38,0.89,1,0.49,0.51


In [45]:
for key, val in code_good.items():
 
    com_name = val[1]
    
    fname = f'df_{com_name}_sel.pkl'
    f_name = directory_for_predict + fname
    df_o = pd.read_pickle(f_name) 

    current_data = df_o.loc[:, 'retail_1':'weekday'] # select columns except targets columns

    last_row = current_data.iloc[[-1]] # get the last row data == previous date data
    
    # locate the model data directory
    directory_model_data = f'../machine_learning/{com_name}/'

    # get the model data filepath
    columns_pkl = directory_model_data + 'best_columns.pkl'
    scaler_pkl = directory_model_data + 'best_scaler.pkl'
    scaler_pkl = directory_model_data + 'scaler_bs_0404_0930_v1.pkl'
    estimator_pkl = directory_model_data + 'best_estimator.pkl'
    estimator_pkl = directory_model_data + 'estimator_bs_0404_0930_v1.pkl'
    result_pkl = directory_model_data + 'best_result.pkl'
    
    # load result data
    result = load_from_pickle(result_pkl)[:-5] 
    precision = result.loc['precision'].iloc[-1]
    
    # load columns data
    real_columns = load_from_pickle(columns_pkl)[:-5] # column 읽기. target columns 5개는 제외
    real_data_df = last_row[real_columns] # select necessary columns
    
    # scale the data
#     scaler = joblib.load(scaler_pkl) # scaler 읽기
    scaler = load_from_pickle(scaler_pkl) # scaler 읽기
    real_scaled = scaler.transform(real_data_df)
    
    # apply the scaled real_data to the model
    print("******", estimator_pkl)
#     estimator = joblib.load(estimator_pkl) # model 읽기
    estimator = load_from_pickle(estimator_pkl) # model 읽기
    model = estimator.best_estimator_

    y_predict = model.predict(real_scaled)
    weight = model.predict_proba(real_scaled)

    df_temp = to_df(last_row.index[-1].date(), com_name, precision, y_predict, weight)
    df_base = pd.concat([df_base, df_temp],axis=1)
    
    print(f'**date: {last_row.index[-1].date()}, {precision:.2f}, {com_name}, 예측: {y_predict}, 가능성:{weight}')
    
# prediciton_list = pd.concat([prediction_list, df_base])
# directory_for_predict = '../data/data_for_ml/predict/'
# fname_p = '0_result.pkl'
# fname_c = '0_result.csv'
# path_p = directory_for_predict + fname_p
# path_c = directory_for_predict + fname_c
# df_base.to_pickle(path_p)
# df_base.to_csv(path_c)

****** ../machine_learning/sec/estimator_bs_0404_0930_v1.pkl
**date: 2023-04-04, 0.93, sec, 예측: [0], 가능성:[[0.5349483 0.4650517]]


In [44]:
prediction_list.head()

,sec_precision,sec_predict,sec_yes,sec_no,naver_precision,naver_predict,naver_yes,naver_no,kakao_precision,kakao_predict,...,lgchemical_yes,lgchemical_no,secpre_precision,secpre_predict,secpre_yes,secpre_no,lgenergy_precision,lgenergy_predict,lgenergy_yes,lgenergy_no
date,,,,,,,,,,,,,,,,,,,,,
2023-04-03,0.93,0,0.52,0.48,1.00,1,0.48,0.52,1.00,0,...,0.50,0.50,1.00,0,0.62,0.38,0.89,1,0.49,0.51


In [28]:
estimator = load_from_pickle(estimator_pkl)

AttributeError: Can't get attribute 'rv_continuous_frozen' on <module 'scipy.stats._distn_infrastructure' from 'c:\\users\\user\\appdata\\local\\programs\\python\\python39\\lib\\site-packages\\scipy\\stats\\_distn_infrastructure.py'>

In [21]:
!pip install --upgrade --user pandas

In [66]:
path_p

'./data/data_for_ml/predict/0_result.pkl'

In [62]:
df_base

,sec_precision,sec_predict,sec_yes,sec_no,naver_precision,naver_predict,naver_yes,naver_no,kakao_precision,kakao_predict,...,lgchemical_yes,lgchemical_no,secpre_precision,secpre_predict,secpre_yes,secpre_no,lgenergy_precision,lgenergy_predict,lgenergy_yes,lgenergy_no
date,,,,,,,,,,,,,,,,,,,,,
2023-04-03,0.93,0,0.52,0.48,1.00,1,0.48,0.52,1.00,0,...,0.50,0.50,1.00,0,0.62,0.38,0.89,1,0.49,0.51


In [43]:
df_t.index = [last_row.index[-1].date()]

In [63]:
path_p

'./data/data_for_ml/predict/0_result.pkl'

In [49]:
df_temp = to_df(last_row.index[-1].date(), com_name, precision, y_predict, weight)

In [53]:
df_temp

,lgenergy_precision,lgenergy_predict,lgenergy_yes,lgenergy_no
date,,,,
2023-04-03,0.89,1,0.49,0.51


In [56]:
df_base = pd.concat([df_base, df_temp],axis=1)

In [57]:
df_base

,lgenergy_precision,lgenergy_predict,lgenergy_yes,lgenergy_no
date,,,,
2023-04-03,0.89,1,0.49,0.51


In [ ]:
pickle['naver']['scaler'] = ['scaler_bs_0403_1409_v1.pkl' ]

In [ ]:
pickle['naver']['estimator'] = ['lgbm_bs_0403_1409_v1.pkl']

In [ ]:
model_data = {}
pickle_code = {}

model_data['columns'] = 'best_columns.pkl'
model_data['scaler'] = 'best_scaler.pkl'
model_data['estimator'] = 'best_estimator.pkl'

pickle_code['naver'] = model_data

In [ ]:
# locate the model data directory
com_name = list(code.values())[0][1]
directory_model_data = f'../machine_learning/{com_name}/'

# get the model data filepath
columns_pkl = directory_model_data + 'columns_bs_0331_2228_101_100%_ver8.pkl'
scaler_pkl = directory_model_data + 'scaler_bs_0331_2228_v8.pkl'
model_pkl = directory_model_data + 'lgbm_bs_0331_2228_v8.pkl'

In [ ]:
# load columns data
real_columns = load_from_pickle(columns_pkl)[:-5] # column 읽기. target columns 5개는 제외
real_data_df = real_data[real_columns] # select necessary columns

In [ ]:
# scale the data
scaler = joblib.load(scaler_pkl) # scaler 읽기
real_scaled = scaler.transform(real_data_df)

In [ ]:
real_scaled

In [ ]:
# apply the scaled real_data to the model
model_ = joblib.load(model_pkl) # model 읽기
model = model_.best_estimator_

y_predict = model.predict(real_scaled)
weight = model.predict_proba(real_scaled)

print(y_predict, weight)

In [ ]:
df_o.iloc[[-8]]['cr_00'].values

In [ ]:
code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy'], 
        '000660' : ['SK하이닉스', 'skhinix'], '207940' : ['삼성바이오로직스', 'ssbio'],
        '006400' : ['삼성SDI', 'sdi'], '051910' : ['LG화학', 'lgchemical'],
        '005935' : ['삼성전자우', 'secpre'], '005380' : ['현대차', 'hyunmotor'],
        '035420' : ['NAVER', 'naver'], '000270' : ['기아','kia'],
        '035720' : ['카카오', 'kakao'], '005490' : ['POSCO홀딩스', 'poscoholding'],
        '105560' : ['KB금융', 'kbbank'], '028260' : ['삼성물산', 'sscnt'],
        '068270' : ['셀트리온', 'celltrion'], '012330' : ['현대모비스', 'mobis'],
        '055550' : ['신한지주', 'shgroup'], '066570' : ['LG전자', 'lgelec'],
        '003670' : ['포스코케미칼', 'poscochemical'], '096770' : ['SK이노베이션', 'skinnovation'],
        '033780' : ['KT&G', 'ktng']}

code = {'005930' : ['삼성전자', 'sec']}

com_name = 'sec'

directory_for_predict = '../data/data_for_ml/predict/'
fname = f'df_{com_name}_sel.pkl'
f_name = directory_for_predict + fname
df_o = pd.read_pickle(f_name) 

df_read = df_o.loc[:, 'retail_1':'weekday'] # select columns except targets columns
real_data = df_read.iloc[[-1]] # get the last row data == previous date data

com_name = list(code.values())[0][1]

directory_model_data = f'../machine_learning/{com_name}/'

# get the model data filepath

columns_pkl = directory_model_data + 'columns_0402_2349_60_93%_ver2.pkl'
scaler_pkl = directory_model_data + 'scaler_0402_2349_v2.pkl'
model_pkl = directory_model_data + 'lgbm_0402_2349_v2.pkl'

real_columns = load_from_pickle(columns_pkl)[:-5] # column 읽기. target columns 5개는 제외
real_data_df = real_data[real_columns] # select necessary columns

scaler = joblib.load(scaler_pkl) # scaler 읽기
real_scaled = scaler.transform(real_data_df)

# apply the scaled real_data to the model
model_ = joblib.load(model_pkl) # model 읽기
model = model_.best_estimator_

y_predict = model.predict(real_scaled)
weight = model.predict_proba(real_scaled)

print(y_predict, weight)

In [ ]:
com_name = 'sec'

directory_for_ml = '../data/data_for_ml/expand_date/'
fname = f'df_{com_name}_sel.pkl'
f_name = directory_for_ml + fname
df_o = pd.read_pickle(f_name) 

In [ ]:
df_read = df_o.loc[:, 'retail_1':'weekday']

In [ ]:
df_f = df_read.iloc[[-1]]

In [ ]:
df_f

In [ ]:
real_columns

In [ ]:
def predict_p(test_target, y_predict_list): 
    ps = precision_score(test_target, y_predict_list)
    rs = recall_score(test_target, y_predict_list)
    fs = f1_score(test_target, y_predict_list)
    roc = roc_auc_score(test_target, y_predict_list)
    collect_list = [ps, rs, fs, roc]
    return collect_list

In [ ]:
def make_df_from_estimator(estimator, num):
    df_t = pd.DataFrame.from_dict(estimator, orient='index')
    df_t.columns = [f'value{num}']
    df_t.index.name = 'parameter'
    return df_t

In [ ]:
def calc_results(model, train_scaled, val_scaled, test_scaled, train_target, val_target, test_target,
                 test_scaled1, test_scaled2, test_scaled3, test_target1, test_target2, test_target3):
# model = lgbmgs.best_estimator_  # 최적의 파라미터로 모델 생성
    y_predict = model.predict(test_scaled)
    result_dict= {}
    result_dict['best_score'] = lgbmgs.best_score_ 
    result_dict['best_index'] = lgbmgs.best_index_
    result_dict['acc_train'] = model.score(train_scaled, train_target)
    result_dict['acc_val'] = model.score(val_scaled, val_target)
    result_dict['acc_test'] = model.score(test_scaled, test_target)
    result_dict['precision'] = precision_score(test_target, y_predict)
    result_dict['recall'] = recall_score(test_target, y_predict)
    result_dict['f1score'] = f1_score(test_target, y_predict)
    result_dict['roc'] = roc_auc_score(test_target, y_predict)
    cm = confusion_matrix(test_target, y_predict)
    result_dict['tn'] = cm[0,0]
    result_dict['fp'] = cm[0,1]
    result_dict['fn'] = cm[1,0]
    result_dict['tp'] = cm[1,1]
    #------------------------------------
    y_predict = model.predict(test_scaled1)
    result_dict['acc_test1'] = model.score(test_scaled1, test_target1)
    result_dict['precision1'] = precision_score(test_target1, y_predict)
    result_dict['recall1'] = recall_score(test_target1, y_predict)
    result_dict['f1score1'] = f1_score(test_target1, y_predict)
    result_dict['roc1'] = roc_auc_score(test_target1, y_predict)
    cm = confusion_matrix(test_target1, y_predict)
    result_dict['tn1'] = cm[0,0]
    result_dict['fp1'] = cm[0,1]
    result_dict['fn1'] = cm[1,0]
    result_dict['tp1'] = cm[1,1]
    #------------------------------------
    y_predict = model.predict(test_scaled2)
    result_dict['acc_test2'] = model.score(test_scaled2, test_target2)
    result_dict['precision2'] = precision_score(test_target2, y_predict)
    result_dict['recall2'] = recall_score(test_target2, y_predict)
    result_dict['f1score2'] = f1_score(test_target2, y_predict)
    result_dict['roc2'] = roc_auc_score(test_target2, y_predict)
    cm = confusion_matrix(test_target2, y_predict)
    result_dict['tn2'] = cm[0,0]
    result_dict['fp2'] = cm[0,1]
    result_dict['fn2'] = cm[1,0]
    result_dict['tp2'] = cm[1,1]
    #------------------------------------
    y_predict = model.predict(test_scaled3)
    result_dict['acc_test3'] = model.score(test_scaled3, test_target3)
    result_dict['precision3'] = precision_score(test_target3, y_predict)
    result_dict['recall3'] = recall_score(test_target3, y_predict)
    result_dict['f1score3'] = f1_score(test_target3, y_predict)
    result_dict['roc3'] = roc_auc_score(test_target3, y_predict)
    cm = confusion_matrix(test_target3, y_predict)
    result_dict['tn3'] = cm[0,0]
    result_dict['fp3'] = cm[0,1]
    result_dict['fn3'] = cm[1,0]
    result_dict['tp3'] = cm[1,1]
    return result_dict

In [ ]:
# save parameters and columns used for the analysis
def save_parameters(iter, com_name, dt, precision, params_o, new_col):
    save_to_pickle(f'./{com_name}/params_{dt}_{round(precision*100):2d}%_ver{iter}.pkl', params_o)
    save_dict_to_csv(f'./{com_name}/params_{dt}_{round(precision*100):2d}%_ver{iter}.pkl', params_o)
    save_to_pickle(f'./{com_name}/columns_{dt}_{len(new_col)}_{round(precision*100):2d}%_ver{iter}.pkl', new_col)
    save_list_to_csv(f'./{com_name}/columns_{dt}_{len(new_col)}_{round(precision*100):2d}%_ver{iter}.pkl', new_col)

In [ ]:
code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy'], 
        '000660' : ['SK하이닉스', 'skhinix'], '207940' : ['삼성바이오로직스', 'ssbio'],
        '006400' : ['삼성SDI', 'sdi'], '051910' : ['LG화학', 'lgchemical'],
        '005935' : ['삼성전자우', 'secpre'], '005380' : ['현대차', 'hyunmotor'],
        '035420' : ['NAVER', 'naver'], '000270' : ['기아','kia'],
        '035720' : ['카카오', 'kakao'], '005490' : ['POSCO홀딩스', 'poscoholding'],
        '105560' : ['KB금융', 'kbbank'], '028260' : ['삼성물산', 'sscnt'],
        '068270' : ['셀트리온', 'celltrion'], '012330' : ['현대모비스', 'mobis'],
        '055550' : ['신한지주', 'shgroup'], '066570' : ['LG전자', 'lgelec'],
        '003670' : ['포스코케미칼', 'poscochemical'], '096770' : ['SK이노베이션', 'skinnovation'],
        '033780' : ['KT&G', 'ktng']}

code = {'005930' : ['삼성전자', 'sec']}

In [ ]:
col_inv1 = ['retail_1', 'foreigner_1', 'institution_1', 'financial_1', 'invtrust_1', 'pension_1', 
#             'privequity_1', 'bank_1', 'insurance_1', 'financeetc_1', 'corporateetc_1', 
            'privequity_1',  'insurance_1', 'corporateetc_1', # bank_1, 'financeetc_1 제외
            'foreigneretc_1']
col_inv2 = ['retail_2', 'foreigner_2', 'institution_2', 'financial_2', 'invtrust_2', 'pension_2',
#             'privequity_2', 'bank_2', 'insurance_2', 'financeetc_2', 'corporateetc_2', 
            'privequity_2', 'insurance_2', 'corporateetc_2', # bank_2, 'financeetc_2 제외
            'foreigneretc_2']
col_his1 = ['open_1', 'high_1', 'low_1', 'close_1', 'vol_1']
col_his2 = ['open_2', 'high_2', 'low_2', 'close_2', 'vol_2']
col_cr = ['weekday', 'cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20']
col_common1 = ["dji_cr", "dji_f_cr", "dxy_cr", "ixic_f_cr", "bond_kor_10_cr", "bond_kor_2_cr", "kosdaq_cr", "kospi_cr", 
         "krw_cr", "ixic_cr", "spx_f_cr", "sox_cr", "spx_cr", "bond_usa_10_cr", "bond_usa_2_cr", "bond_usa_3m_cr", 
         "vix_cr", "wti_cr", "spsy_cr", "spny_cr", "spxhc_cr", "splrcd_cr", "splrci_cr", "splrcu_cr", "splrcs_cr",
         "splrct_cr", "splrcl_cr", "splrcm_cr", "ixbk_cr", "ixfn_cr", "ixid_cr", "ixis_cr", "ixk_cr", "ixtr_cr",
         "ixut_cr", "nbi_cr", "bkx_cr"]
col_common2 = ["dji_cr_2", "dji_f_cr_2", "dxy_cr_2", "ixic_f_cr_2", "bond_kor_10_cr_2", "bond_kor_2_cr_2", "kosdaq_cr_2", "kospi_cr_2",
         "krw_cr_2", "ixic_cr_2", "spx_f_cr_2", "sox_cr_2", "spx_cr_2", "bond_usa_10_cr_2", "bond_usa_2_cr_2", "bond_usa_3m_cr_2",
         "vix_cr_2", "wti_cr_2", "spsy_cr_2", "spny_cr_2", "spxhc_cr_2", "splrcd_cr_2", "splrci_cr_2", "splrcu_cr_2",
         "splrcs_cr_2", "splrct_cr_2", "splrcl_cr_2", "splrcm_cr_2", "ixbk_cr_2", "ixfn_cr_2", "ixid_cr_2",
         "ixis_cr_2", "ixk_cr_2", "ixtr_cr_2", "ixut_cr_2", "nbi_cr_2", "bkx_cr_2"]
new_col = col_inv1 + col_common1 + col_his1 + col_inv2 + col_common2 + col_his2 + col_cr

# bank, financeetc는 결측치가 많아서 사용하지 않음.
# df.drop(['bank_1', 'bank_2', 'financeetc_1', 'financeetc_2'], axis=1, inplace=True)   

In [ ]:
# 최초의 empty df 생성
df_base = pd.DataFrame(pd.Series([],dtype=pd.StringDtype(), name='parameter')).set_index('parameter')
iter = 0

In [ ]:
com_name = 'sec'

directory_for_ml = '../data/data_for_ml/expand_date/'
fname = f'df_{com_name}_sel.pkl'
f_name = directory_for_ml + fname
df_o = pd.read_pickle(f_name) 

In [ ]:
new_col = load_from_pickle('./sec/columns_0323_1641_50_81%_ver8.pkl')

In [ ]:
# 반복 작업시 여기서 부터 진행 (feature importance로 선정된 새로운 column으로)

df = df_o[new_col]  # 새롭게 선정된 column으로 진행

# train, val,: 8, test: 2
split_ratio = 0.8
split_n = int(len(df)*split_ratio)

test_interval = int((len(df) - split_n)/3)
data = df.iloc[:split_n, :-5]
target = df.iloc[:split_n, -5]
test_input = df.iloc[split_n:, :-5]
test_target = df.iloc[split_n:, -5]
test_input1 = df.iloc[split_n:split_n+test_interval, :-5]
test_input2 = df.iloc[split_n+test_interval: split_n+test_interval*2, :-5]
test_input3 = df.iloc[split_n+test_interval*2:, :-5]
test_target1 = df.iloc[split_n:split_n+test_interval, -5]
test_target2 = df.iloc[split_n+test_interval: split_n+test_interval*2, -5]
test_target3 = df.iloc[split_n+test_interval*2:, -5]

train_input, val_input, train_target, val_target = \
     train_test_split(data, target, random_state=42, test_size=0.2, stratify=target)

scaler = StandardScaler()
scaler.fit(train_input)
train_scaled = scaler.transform(train_input)
val_scaled = scaler.transform(val_input)
test_scaled = scaler.transform(test_input)
test_scaled1 = scaler.transform(test_input1)
test_scaled2 = scaler.transform(test_input2)
test_scaled3 = scaler.transform(test_input3)

In [ ]:
param_base = {'boosting_type' : ['gbdt'], # ['gbdt', dart', 'goss'], # dart : 신경망의 드롭아웃을 적용시킨 방법, 
              'num_leaves' :  [6, 7, 8,], # 두번째로 중요, num_leaves는 작은 데이터면 작은 숫자로
              'learning_rate' :  [0.001, 0.0015, 0.0017, 0.002, 0.0025],
#               'max_delta_step' :  [0.4, 0.5, 0.6],
              'n_estimators' :  [ 1100, 1200, 1300, 1400],
              'colsample_bytree' :  [ 0.5, 0.6, 0.7, 0.8], # = feature fraction, column sampling. 위의 subsample과 같이 튜닝.
              'subsample' : [1], #  [0.1, 0.2, 0.3, 0.4], # 세번째로 중요. = bagging fraction, row sampling. 아래 colsample_bytree과 같이 튜닝.
              'max_depth' :  [-1], # 가장 먼저 튜닝 필요 -1이 default (무한깊이) 일반적으로 default가 가장 좋음.
              'objective': ['binary'],
              'metric': ['binary_logloss'],
              'scale_pos_weight': [1.0, 1.5, 2], # posiive  증가, class imbalance 경감, scale_pos_weight > 0.0, default=1.0. 
#               'min_child_samples' : [30],
#               'lambda_l1': [0, 5], # default 0
#               'lambda_l2': [0, 5], #default 0
              }

In [ ]:
param_grid = {
#     'cv' : 7,
    'cv' : None,
    'scoring' : None,
#     'scoring' : 'precision',
#     'scoring' : 'accuracy',
    'num_col' : len(new_col)
             }

In [ ]:
param_base = load_from_pickle('./sec/params_0323_1641_81%_ver8.pkl') # 읽고 수정하기

In [ ]:
param_base = {'boosting_type': ['gbdt'],
 'num_leaves': [6,],
 'learning_rate': [0.0015,],
 'n_estimators': [1300],
 'colsample_bytree': [0.6,],
 'subsample': [1],
 'max_depth': [-1],
 'objective': ['binary'],
 'metric': ['binary_logloss'],
 'scale_pos_weight': [1.5]}

In [ ]:
params_o = param_base.copy()

In [ ]:
# directory가 없으면 만드는 과정
if not os.path.exists(com_name):
    os.makedirs(com_name)
    
while True:
    iter = iter + 1

    lgbm = None
    lgbmgs = None

    lgbm = lightgbm.LGBMClassifier(random_state=42)

    lgbmgs = GridSearchCV(estimator = lgbm,
                          param_grid = params_o,
#                           cv = 10, # StratifiedKFold us default for binary or multiclass
#                           scoring = 'precision', 
#                           scoring = 'accuracy', 
#                           scoring = ['accuracy', 'precision'], # refit 사용해야 함. 고로 사용하지 않음.
                          cv = param_grid['cv'],
                          scoring = param_grid['scoring'],
                          error_score='raise',
                          verbose = 1,
                          n_jobs=-1, # 자동 검색 적용
                         )
                          
    print("*** after lgbmgs ******")
    lgbmgs.fit(train_scaled, train_target, eval_metric = 'logloss') 
#     lgbmgs.fit(train_scaled, train_target, eval_metric = 'logloss', eval_set = (val_scaled, val_target)) 
    # eval_set가 있어야 "early_stopping_rounds"를 사용할 수 있음.

# save model
    stamp = datetime.datetime.today().isoformat() # 파일명 끝에 생성날짜 시간 추가
    dt = re.sub(r'[-:T]', '', stamp[5:16])
    dt = f'{dt[:4]}_{dt[4:]}'
    joblib.dump(lgbmgs, f'./{com_name}/lgbm_{dt}_v{iter}.pkl') # gridsearchcv 저장
    joblib.dump(scaler, f'./{com_name}/scaler_{dt}_v{iter}.pkl') # scaler 저장
    
    df_estimator = make_df_from_estimator(lgbmgs.best_estimator_.get_params(), iter)
    result_dict = calc_results(lgbmgs.best_estimator_, 
                               train_scaled, val_scaled, test_scaled,  
                               train_target, val_target, test_target,
                               test_scaled1, test_scaled2, test_scaled3,  
                               test_target1, test_target2, test_target3
                              )
    
    df_result = make_df_from_estimator(result_dict, iter)
    df_grid = make_df_from_estimator(param_grid, iter)
    df_concat = pd.concat([df_estimator, df_grid, df_result])

    df_base = pd.merge(df_base,df_concat, how='outer', left_index=True, right_index=True)
    
#  4가지 조건이 만족되면 break하고 완료
    val_test = df_concat.loc['acc_val'].iloc[0]
    acc_test = df_concat.loc['acc_test'].iloc[0]
    precision = df_concat.loc['precision'].iloc[0]
    f1score = df_concat.loc['f1score'].iloc[0]
    
    if (val_test >= 0.75 ) & (acc_test > 0.75) & (precision >= 0.8) & (f1score >= 0.6) :
        df_base.to_csv(f'./{com_name}/lgbm_df_{dt}_{round(precision*100):2d}%_ver{iter}.csv')
        df_base.to_pickle(f'./{com_name}/lgbm_df_{dt}_{round(precision*100):2d}%_ver{iter}.pkl')
#         params_o = make_new_parameter(params_o, df_concat)
#         save_to_pickle(f'./{com_name}/params_{dt}_{round(precision*100):2d}%_ver{iter}.pkl', params_o)
#         save_dict_to_csv(f'./{com_name}/params_{dt}_{round(precision*100):2d}%_ver{iter}.pkl', params_o)
#         save_to_pickle(f'./{com_name}/columns_{dt}_{len(new_col)}_{round(precision*100):2d}%_ver{iter}.pkl', new_col)
#         save_list_to_csv(f'./{com_name}/columns_{dt}_{len(new_col)}_{round(precision*100):2d}%_ver{iter}.pkl', new_col)
        save_parameters(iter, com_name, dt, precision, params_o, new_col)
        break
    if iter >= 1 : 
        df_base.to_csv(f'./{com_name}/lgbm_df_{dt}_{round(precision*100):2d}%_ver{iter}.csv')
        df_base.to_pickle(f'./{com_name}/lgbm_df_{dt}_{round(precision*100):2d}%_ver{iter}.pkl')
#         params_o = make_new_parameter(params_o, df_concat)
#         save_to_pickle(f'./{com_name}/params_{dt}_{round(precision*100):2d}%_ver{iter}.pkl', params_o)
#         save_dict_to_csv(f'./{com_name}/params_{dt}_{round(precision*100):2d}%_ver{iter}.pkl', params_o)
#         save_to_pickle(f'./{com_name}/columns_{dt}_{len(new_col)}_{round(precision*100):2d}%_ver{iter}.pkl', new_col)
#         save_list_to_csv(f'./{com_name}/columns_{dt}_{len(new_col)}_{round(precision*100):2d}%_ver{iter}.pkl', new_col)
        save_parameters(iter, com_name, dt, precision, params_o, new_col)
        break
    print("******* No.{}  Process is Done! ********".format(iter))
#     params_o = make_new_parameter(params_o, df_concat)
#     save_to_pickle(f'./{com_name}/params_{dt}_{round(precision*100):2d}%_ver{iter}.pkl', params_o)
#     save_dict_to_csv(f'./{com_name}/params_{dt}_{round(precision*100):2d}%_ver{iter}.pkl', params_o)
#     save_to_pickle(f'./{com_name}/columns_{dt}_{len(new_col)}_{round(precision*100):2d}%_ver{iter}.pkl', new_col)
#     save_list_to_csv(f'./{com_name}/columns_{dt}_{len(new_col)}_{round(precision*100):2d}%_ver{iter}.pkl', new_col)
    save_parameters(iter, com_name, dt, precision, params_o, new_col)
    
print("**** End of Process ****")
# save model, save df, stoppping 기준 수립
# 일단위로 정확도 측정, 정확도, 정밀도?


In [ ]:
df_base.head(60)

칼럼수를 90으로 줄이니까, precision이 90%으로 향상됨.

In [ ]:
print(lgbmgs.best_estimator_)

In [ ]:
model = lgbmgs.best_estimator_
feature_df = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), 
                      index=data.columns, columns=['importance']).sort_values(by='importance', 
                                                                              ascending=False)

In [ ]:
len(feature_df)

In [ ]:
feature_df.head(90)

In [ ]:
new_col = list(feature_df.index[:80]) +  ['cr_00', 'cr_05', 'cr_10', 'cr_15', 'cr_20']

In [ ]:
model.booster_.feature_importance(importance_type='gain')

In [ ]:
model.booster_.feature_importance(importance_type='split')
# 큰 특징을 가지는 feature는 tree상위레벨에서 적게 사용됨.

In [ ]:
# 사용된 column을 불러와서 신규column 지정하여 최적의 column 찾기에 사용
directory_for_ml = '../machine_learning/sec/'
fname = f'columns_ver1_0323_1535_72_0.74%.pkl'
f_name = directory_for_ml + fname
new_col = load_from_pickle(f_name)
new_col =  new_col[:61]+new_col[-5:] # 원하는 column 선정

1. 정밀도, f1-score, 
2. confusion matrix ((1,1), (2,2), 두개가 큰 비중이면 good, (1,2)은 틀린것을 맞다라고 구분, (2,1)은 맞는 것을 틀린 것이다 라고 결정하는 항목) 따라서
    (2,2) -> (1,2) -> (1,1)로 확인하고. <br>
    (1,2)가 크면 모델 제외 (정밀도(precision = TP / (TP + FP) )가 높아야 함. 낮으면 손해를 보게 됨.), <br>
    재현율(Recall = TP / (TP + FN) ) 은 손해를 끼치지는 않음.
    
<img src="https://raw.githubusercontent.com/fasthill/My-gist/main/data/picture/confusion_matrix.png" width="800"/> <br>

In [ ]:
directory_for_ml = '../data/data_for_ml/expand_date/'
ss = pd.read_csv(directory_for_ml+'df_sec_sel.csv', index_col=0)

In [ ]:
ss